<a href="https://colab.research.google.com/github/schizohub/ML_projects/blob/main/Customer_churn_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import torch
import plotly.express as px

In [22]:
url="https://github.com/anvarnarz/praktikum_datasets/blob/main/E-Commerce-Dataset.xlsx?raw=true"
df = pd.read_excel(url, sheet_name="E Comm",index_col=0)
df.head()

,Churn,Tenure,PreferredLoginDevice,CityTier,WarehouseToHome,PreferredPaymentMode,Gender,HourSpendOnApp,NumberOfDeviceRegistered,PreferedOrderCat,SatisfactionScore,MaritalStatus,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,OrderCount,DaySinceLastOrder,CashbackAmount
CustomerID,,,,,,,,,,,,,,,,,,,
50001,1,4.0,Mobile Phone,3,6.0,Debit Card,Female,3.0,3,Laptop & Accessory,2,Single,9,1,11.0,1.0,1.0,5.0,159.93
50002,1,NaN,Phone,1,8.0,UPI,Male,3.0,4,Mobile,3,Single,7,1,15.0,0.0,1.0,0.0,120.90
50003,1,NaN,Phone,1,30.0,Debit Card,Male,2.0,4,Mobile,3,Single,6,1,14.0,0.0,1.0,3.0,120.28
50004,1,0.0,Phone,3,15.0,Debit Card,Male,2.0,4,Laptop & Accessory,5,Single,8,0,23.0,0.0,1.0,3.0,134.07
50005,1,0.0,Phone,1,12.0,CC,Male,NaN,3,Mobile,5,Single,3,0,11.0,1.0,1.0,3.0,129.60


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5630 entries, 50001 to 55630
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Churn                        5630 non-null   int64  
 1   Tenure                       5366 non-null   float64
 2   PreferredLoginDevice         5630 non-null   object 
 3   CityTier                     5630 non-null   int64  
 4   WarehouseToHome              5379 non-null   float64
 5   PreferredPaymentMode         5630 non-null   object 
 6   Gender                       5630 non-null   object 
 7   HourSpendOnApp               5375 non-null   float64
 8   NumberOfDeviceRegistered     5630 non-null   int64  
 9   PreferedOrderCat             5630 non-null   object 
 10  SatisfactionScore            5630 non-null   int64  
 11  MaritalStatus                5630 non-null   object 
 12  NumberOfAddress              5630 non-null   int64  
 13  Complain     

In [33]:
y = df.Churn
y.to_numpy()

array([1, 1, 1, ..., 0, 0, 0])

In [5]:
df.describe()

,Churn,Tenure,CityTier,WarehouseToHome,HourSpendOnApp,NumberOfDeviceRegistered,SatisfactionScore,NumberOfAddress,Complain,OrderAmountHikeFromlastYear,CouponUsed,OrderCount,DaySinceLastOrder,CashbackAmount
count,5630.000000,5366.000000,5630.000000,5379.000000,5375.000000,5630.000000,5630.000000,5630.000000,5630.000000,5365.000000,5374.000000,5372.000000,5323.000000,5630.000000
mean,0.168384,10.189899,1.654707,15.639896,2.931535,3.688988,3.066785,4.214032,0.284902,15.707922,1.751023,3.008004,4.543491,177.223030
std,0.374240,8.557241,0.915389,8.531475,0.721926,1.023999,1.380194,2.583586,0.451408,3.675485,1.894621,2.939680,3.654433,49.207036
min,0.000000,0.000000,1.000000,5.000000,0.000000,1.000000,1.000000,1.000000,0.000000,11.000000,0.000000,1.000000,0.000000,0.000000
25%,0.000000,2.000000,1.000000,9.000000,2.000000,3.000000,2.000000,2.000000,0.000000,13.000000,1.000000,1.000000,2.000000,145.770000
50%,0.000000,9.000000,1.000000,14.000000,3.000000,4.000000,3.000000,3.000000,0.000000,15.000000,1.000000,2.000000,3.000000,163.280000
75%,0.000000,16.000000,3.000000,20.000000,3.000000,4.000000,4.000000,6.000000,1.000000,18.000000,2.000000,3.000000,7.000000,196.392500
max,1.000000,61.000000,3.000000,127.000000,5.000000,6.000000,5.000000,22.000000,1.000000,26.000000,16.000000,16.000000,46.000000,324.990000


# Data cleaning

# 1.1 Filling NaNs

In [6]:
class DataClean():

    def __init__(self, df):
        self.df = df

    def dataFilling(self):
       self.df.fillna(method='ffill', inplace=True)
    
    def fillObjectCols(self):
        from sklearn.preprocessing import LabelEncoder 
        le = LabelEncoder()
        for i in self.df.columns:
            if type(self.df[i].values[0])==str:
                self.df[i] = le.fit_transform(self.df[i])
       
    def selectFeatures(self, standardize=False):
        from sklearn.feature_selection import SelectKBest, chi2

        X, y = self.df.drop('Churn', axis=1).to_numpy(), self.df['Churn'].values
        X_new = SelectKBest(chi2, k=10).fit_transform(X, y)
        if standardize:
            from sklearn.preprocessing import StandardScaler
            std = StandardScaler()
            columns_selected = SelectKBest(chi2, k=10).fit(X,y).get_feature_names_out(self.df.drop('Churn', axis=1).columns)
            X_new = pd.DataFrame(std.fit_transform(X_new))
            X_new.columns = columns_selected
            return X_new
        return X_new

    def normalize_outliers_IQR(self,col,inplace=False):
        q1=self.df[col].quantile(0.25)
        q3=self.df[col].quantile(0.75)
        IQR=q3-q1
        outliers = self.df[col][((self.df[col]<(q1-1.5*IQR)) | (self.df[col]>(q3+1.5*IQR)))]
        if len(outliers)>50:
            upper = self.df[col][~(self.df[col]>(q3+1.5*IQR))].max()
            lower = self.df[col][~(self.df[col]<(q1-1.5*IQR))].min()
            new = np.where(self.df[col] > upper,self.df[col].mean(),np.where(self.df[col] < lower,self.df[col].mean(),self.df[col]))
            if inplace:
                return new
        return self.df[col]
        
    def PCA_reduce(self, scaled_data):
        from sklearn.decomposition import PCA
        pca = PCA(n_components = 5)
        pca.fit(scaled_data)
        data_pca = pca.transform(scaled_data)
        data_pca = pd.DataFrame(data_pca,columns=['PC1','PC2','PC3','PC4','PC5'])
        return data_pca

In [7]:
cleaner = DataClean(df)
cleaner.dataFilling()
cleaner.fillObjectCols()
X_version1 = cleaner.selectFeatures()
X_version2 = cleaner.selectFeatures(standardize=True)
data_copied = df.copy()
for col in df.drop('Churn', axis=1).columns:
    data_copied[col] = cleaner.normalize_outliers_IQR(col, inplace=True)
new_data = DataClean(data_copied)
data_copied = new_data.selectFeatures(standardize=True)
X_version3 = cleaner.PCA_reduce(data_copied)


In [8]:
X_version2

,Tenure,CityTier,WarehouseToHome,NumberOfDeviceRegistered,PreferedOrderCat,SatisfactionScore,MaritalStatus,Complain,DaySinceLastOrder,CashbackAmount
0,-0.720360,1.469771,-1.100218,-0.672900,-0.261904,-0.772992,1.251898,1.584290,0.119582,-0.351465
1,-0.720360,-0.715286,-0.872187,0.303750,0.446658,-0.048392,1.251898,1.584290,-1.247697,-1.144715
2,-0.720360,-0.715286,1.636151,0.303750,0.446658,-0.048392,1.251898,1.584290,-0.427330,-1.157316
3,-1.187504,1.469771,-0.074080,0.303750,-0.261904,1.400807,1.251898,-0.631198,-0.427330,-0.877047
4,-1.187504,-0.715286,-0.416126,-0.672900,0.446658,1.400807,1.251898,-0.631198,-0.427330,-0.967895
...,...,...,...,...,...,...,...,...,...,...
5625,-0.019644,-0.715286,1.636151,-1.649551,-0.261904,-1.497592,-0.253481,-0.631198,-0.153874,-0.538854
5626,0.330714,-0.715286,-0.302110,1.280401,-1.679027,1.400807,-0.253481,-0.631198,-0.153874,0.969195
5627,-1.070718,-0.715286,-0.530141,-1.649551,-0.261904,0.676207,-0.253481,1.584290,-0.153874,0.186920
5628,1.498574,1.469771,-0.758172,1.280401,-0.261904,0.676207,-0.253481,-0.631198,1.213406,0.034083


In [9]:
X_version1[:,7]

array([1., 1., 1., ..., 1., 0., 0.])

In [19]:
X_version3

,PC1,PC2,PC3,PC4,PC5,Churn
0,0.151289,0.506556,1.920840,-1.128406,-0.703275,1
1,-1.906359,-0.709657,1.863250,-0.230797,-0.339230,1
2,-1.524966,0.173233,2.063917,0.995720,0.388808,1
3,-0.656849,0.249964,0.761551,-2.279628,1.661362,1
4,-1.879534,-0.042499,-0.510350,-1.026752,0.747913,1
...,...,...,...,...,...,...
5625,-0.078189,-0.231298,0.811645,0.606796,-0.492091,0
5626,2.055958,-0.903305,-0.294912,-0.594302,1.018968,0
5627,-0.193415,0.420452,0.090254,0.974428,0.375929,0
5628,1.566754,-0.040257,-1.295057,-0.776296,0.074434,0


In [11]:
cols = X_version2.columns
cols

Index(['Tenure', 'CityTier', 'WarehouseToHome', 'NumberOfDeviceRegistered',
       'PreferedOrderCat', 'SatisfactionScore', 'MaritalStatus', 'Complain',
       'DaySinceLastOrder', 'CashbackAmount'],
      dtype='object')

In [12]:
X_version1 = pd.DataFrame(X_version1, columns=cols)
X_version3['Churn'] = y.values
X_version2['Churn'] = y.values
X_version1['Churn'] = y.values

In [13]:
X_version2.head()

,Tenure,CityTier,WarehouseToHome,NumberOfDeviceRegistered,PreferedOrderCat,SatisfactionScore,MaritalStatus,Complain,DaySinceLastOrder,CashbackAmount,Churn
0,-0.720360,1.469771,-1.100218,-0.67290,-0.261904,-0.772992,1.251898,1.584290,0.119582,-0.351465,1
1,-0.720360,-0.715286,-0.872187,0.30375,0.446658,-0.048392,1.251898,1.584290,-1.247697,-1.144715,1
2,-0.720360,-0.715286,1.636151,0.30375,0.446658,-0.048392,1.251898,1.584290,-0.427330,-1.157316,1
3,-1.187504,1.469771,-0.074080,0.30375,-0.261904,1.400807,1.251898,-0.631198,-0.427330,-0.877047,1
4,-1.187504,-0.715286,-0.416126,-0.67290,0.446658,1.400807,1.251898,-0.631198,-0.427330,-0.967895,1


In [14]:
X_version1.to_csv("Version1_churn.csv")
X_version2.to_csv("Version2_churn.csv")
X_version3.to_csv("Version3_churn.csv")

# Data Preparing

In [15]:
import requests
from pathlib import Path 

if Path('helper_functions.py').is_file():
    print("File already exists")
else:
    print("Download them")
    request = requests.get("https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/helper_functions.py")
    with open("helper_functions.py", "wb") as f:
        f.write(request.content)
from helper_functions import plot_predictions, plot_decision_boundary

Download them


In [72]:
from torch import nn

device =  'cuda' if torch.cuda.is_available() else 'cpu'
# device
X2 = torch.from_numpy(X_version2.drop('Churn', axis=1).to_numpy()).type(torch.float)
X3 = torch.from_numpy(X_version3.drop('Churn', axis=1).to_numpy()).type(torch.float)
# y = torch.from_numpy(y).type(torch.float)

In [73]:
X1_train, X1_test, y1_train, y1_test = train_test_split(X2, y, test_size=0.2, random_state=42)
X2_train, X2_test, y2_train, y2_test = train_test_split(X3, y, test_size=0.2, random_state=42)
X1_train, X1_test = X1_train.to(device), X1_test.to(device)
X2_train, X2_test = X2_train.to(device), X2_test.to(device)
y1_train, y1_test = y1_train.to(device), y1_test.to(device)
y2_train, y2_test = y2_train.to(device), y2_test.to(device)


# Modelling (Neural Network)

In [84]:
class NNClassifier(nn.Module):
    def __init__(self, input_features,output_features, hidden_units):
        super().__init__()
        self.layer1 = nn.Linear(in_features=input_features, out_features=hidden_units)
        self.layer2 = nn.Linear(in_features=hidden_units,out_features=hidden_units)
        self.layer3 = nn.Linear(in_features=hidden_units,out_features=output_features)

    def forward(self,x):
        return self.layer3(self.layer2(self.layer1(x)))

def accuracy_func(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item()
    acc = (correct/len(y_pred))*100
    return acc

In [75]:
X1_train[0], X2_train[0]

(tensor([-1.0707, -0.7153, -0.4161,  0.3038,  1.1552,  0.6762,  1.2519,  1.5843,
         -0.4273, -0.5578], device='cuda:0'),
 tensor([-1.8752,  0.2649,  0.6761,  0.4138,  0.0330], device='cuda:0'))

In [85]:
modelv0 = NNClassifier(10, 1,  128).to(device)
modelv1 = NNClassifier(5,1,128).to(device)

In [86]:
loss_fn = nn.BCEWithLogitsLoss()

optimizerv0 = torch.optim.SGD(params=modelv0.parameters(),
                            lr=0.3)
optimizerv1 = torch.optim.SGD(params=modelv1.parameters(),
                              lr=0.3)
optimizerv0_0 = torch.optim.Adam(params=modelv0.parameters(),
                                 lr=0.1)
optimizerv1_0 = torch.optim.Adam(params=modelv1.parameters(),
                                 lr=0.1)

In [82]:
modelv0.parameters()

<generator object Module.parameters at 0x7f87c68c4d60>

In [87]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

def tune(model,
         optimizer,
         x_train,
         y_train,
         x_test, 
         y_test, 
         n_epochs=100):
    for epoch in range(n_epochs):
        model.train()

        y_logits = model(x_train).squeeze()
        y_pred = torch.round(torch.sigmoid(y_logits))

        loss = loss_fn(y_logits, y_train)

        acc = accuracy_func(y_train, y_pred)

        optimizer.zero_grad()

        loss.backward()

        optimizer.step()

        model.eval()

        with torch.inference_mode():
            test_logits = model(x_test).squeeze()
            test_pred = torch.round(torch.sigmoid(test_logits))

            test_loss = loss_fn(test_logits, y_test)

            test_acc = accuracy_func(y_test, test_pred)

        if epoch%100==0:
            print(f"Epoch: {epoch}| loss: {loss:.4f}, acc: {acc:.2f}% | test_loss: {test_loss:.4f}, test_acc: {test_acc:.2f}%")

In [88]:
# tune(modelv0,optimizerv0,  X1_train,y1_train, X1_test, y1_test, n_epochs=1000)
tune(modelv0, optimizerv0_0, X1_train, y1_train, X1_test, y1_test, n_epochs=1000)

Epoch: 0| loss: 0.7159, acc: 48.85% | test_loss: 0.8939, test_acc: 85.26%
Epoch: 100| loss: 0.3268, acc: 87.19% | test_loss: 0.3129, test_acc: 88.19%
Epoch: 200| loss: 0.3266, acc: 87.14% | test_loss: 0.3132, test_acc: 88.10%
Epoch: 300| loss: 0.3266, acc: 87.21% | test_loss: 0.3132, test_acc: 88.19%
Epoch: 400| loss: 0.3266, acc: 87.14% | test_loss: 0.3132, test_acc: 88.10%
Epoch: 500| loss: 0.3266, acc: 87.14% | test_loss: 0.3132, test_acc: 88.10%
Epoch: 600| loss: 0.3266, acc: 87.14% | test_loss: 0.3132, test_acc: 88.10%
Epoch: 700| loss: 0.3266, acc: 87.14% | test_loss: 0.3132, test_acc: 88.10%
Epoch: 800| loss: 0.3266, acc: 87.14% | test_loss: 0.3132, test_acc: 88.10%
Epoch: 900| loss: 0.3266, acc: 87.14% | test_loss: 0.3132, test_acc: 88.10%


In [89]:
# tune(modelv1,optimizerv1, X2_train, y2_train, X2_test, y2_test, n_epochs=1000)
tune(modelv1, optimizerv1_0, X2_train, y2_train, X2_test, y2_test, n_epochs=1000)

Epoch: 0| loss: 0.6518, acc: 65.59% | test_loss: 2.2041, test_acc: 83.93%
Epoch: 100| loss: 0.3632, acc: 85.66% | test_loss: 0.3572, test_acc: 86.32%
Epoch: 200| loss: 0.3626, acc: 85.70% | test_loss: 0.3549, test_acc: 86.23%
Epoch: 300| loss: 0.3626, acc: 85.70% | test_loss: 0.3549, test_acc: 86.23%
Epoch: 400| loss: 0.3626, acc: 85.70% | test_loss: 0.3549, test_acc: 86.23%
Epoch: 500| loss: 0.3626, acc: 85.70% | test_loss: 0.3549, test_acc: 86.23%
Epoch: 600| loss: 0.3626, acc: 85.70% | test_loss: 0.3549, test_acc: 86.23%
Epoch: 700| loss: 0.3626, acc: 85.70% | test_loss: 0.3549, test_acc: 86.23%
Epoch: 800| loss: 0.3627, acc: 85.83% | test_loss: 0.3560, test_acc: 85.97%
Epoch: 900| loss: 0.3626, acc: 85.75% | test_loss: 0.3549, test_acc: 86.23%


In [91]:
# plt.figure(figsize=(10,7))
# plt.subplot(2,1,1)
# plt.title("Train version 1")
# plot_decision_boundary(modelv0, X1_train, y1_train)
# plt.subplot(2,1,2)
# plt.title("Test version 1")
# plot_decision_boundary(modelv0, X1_test, y1_test)
# plt.subplot(2,2,1)
# plt.title("Train version 2")
# plot_decision_boundary(modelv1, X2_train, y2_train)
# plt.subplot(2,2,2)
# plt.title("Test version 2")
# plot_decision_boundary(modelv1, X2_test, y2_test)

In [138]:
# !pip install torchmetrics

In [133]:
class NNClassifierV1(nn.Module):
    def __init__(self, input_features, output_features, hidden_units):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=output_features)
        )
    
    def forward(self,x):
        return self.layer_stack(x)

    



In [134]:
modelv0_0 = NNClassifierV1(10, 1 ,256).to(device)
modelv1_1 = NNClassifierV1(5, 1, 256).to(device)


optimizerv0 = torch.optim.SGD(params=modelv0_0.parameters(),
                            lr=0.3)
optimizerv1 = torch.optim.SGD(params=modelv1_1.parameters(),
                              lr=0.3)
optimizerv0_0 = torch.optim.Adam(params=modelv0_0.parameters(),
                                 lr=0.1)
optimizerv1_0 = torch.optim.Adam(params=modelv1_1.parameters(),
                                 lr=0.1)

In [135]:
tune(modelv0_0,optimizerv0_0, X1_train, y1_train, X1_test, y1_test, n_epochs=1000)

Epoch: 0| loss: 0.6500, acc: 81.93% | test_loss: 21.6610, test_acc: 83.57%
Epoch: 100| loss: 0.1881, acc: 94.98% | test_loss: 0.2613, test_acc: 91.83%
Epoch: 200| loss: 0.1813, acc: 95.14% | test_loss: 0.3775, test_acc: 88.99%
Epoch: 300| loss: 0.1439, acc: 96.65% | test_loss: 0.2686, test_acc: 92.81%
Epoch: 400| loss: 0.1392, acc: 96.76% | test_loss: 0.2840, test_acc: 92.90%
Epoch: 500| loss: 0.1383, acc: 96.78% | test_loss: 0.2901, test_acc: 93.25%
Epoch: 600| loss: 0.1378, acc: 96.78% | test_loss: 0.2973, test_acc: 93.43%
Epoch: 700| loss: 0.1376, acc: 96.78% | test_loss: 0.3006, test_acc: 93.16%
Epoch: 800| loss: 0.1376, acc: 96.78% | test_loss: 0.3069, test_acc: 93.07%
Epoch: 900| loss: 0.1375, acc: 96.78% | test_loss: 0.3124, test_acc: 92.90%


In [136]:
tune(modelv1_1,optimizerv1_0, X2_train, y2_train, X2_test, y2_test, n_epochs=1000)


Epoch: 0| loss: 0.7208, acc: 30.37% | test_loss: 21.1233, test_acc: 83.57%
Epoch: 100| loss: 0.1857, acc: 92.21% | test_loss: 0.2936, test_acc: 89.52%
Epoch: 200| loss: 0.1312, acc: 93.81% | test_loss: 0.3236, test_acc: 88.99%
Epoch: 300| loss: 0.2035, acc: 91.05% | test_loss: 0.3060, test_acc: 89.88%
Epoch: 400| loss: 0.1232, acc: 94.49% | test_loss: 0.3758, test_acc: 89.08%
Epoch: 500| loss: 0.0620, acc: 97.80% | test_loss: 0.3701, test_acc: 91.21%
Epoch: 600| loss: 0.0407, acc: 98.96% | test_loss: 0.4092, test_acc: 91.83%
Epoch: 700| loss: 0.0193, acc: 99.60% | test_loss: 0.5338, test_acc: 92.45%
Epoch: 800| loss: 0.0148, acc: 99.73% | test_loss: 0.5861, test_acc: 92.27%
Epoch: 900| loss: 0.0128, acc: 99.64% | test_loss: 0.6353, test_acc: 91.65%


In [159]:
class NNClassifierV2(nn.Module):
    def __init__(self, input_features, output_features, hidden_units):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Linear(in_features=input_features, out_features=hidden_units),
            nn.Sigmoid(),
            nn.Linear(in_features=hidden_units, out_features=hidden_units),
            # nn.Sigmoid(),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=output_features)
        )
    
    def forward(self, x):
        return self.layer_stack(x)

In [160]:
modelv0_1 = NNClassifierV2(10, 1, 128).to(device)
modelv1_2 = NNClassifierV2(5,1,128).to(device)

optimizerv0_1 = torch.optim.SGD(params=modelv0_1.parameters(),
                                lr=0.1)
optimizerv1_1 = torch.optim.SGD(params=modelv1_2.parameters(),
                                lr=0.1)
optimizerv0_2 = torch.optim.Adam(params=modelv0_1.parameters(),
                                 lr=0.3)
optimizerv1_2 = torch.optim.Adam(params=modelv1_2.parameters(),
                                 lr=0.3)

In [161]:
tune(modelv0_1, optimizerv0_2, X1_train, y1_train, X1_test, y1_test, n_epochs=1000)

Epoch: 0| loss: 0.7162, acc: 17.32% | test_loss: 47.5156, test_acc: 83.57%
Epoch: 100| loss: 0.3395, acc: 85.17% | test_loss: 0.3542, test_acc: 85.17%
Epoch: 200| loss: 0.2306, acc: 89.43% | test_loss: 0.2966, test_acc: 88.90%
Epoch: 300| loss: 0.1453, acc: 93.61% | test_loss: 0.2867, test_acc: 91.56%
Epoch: 400| loss: 0.0818, acc: 96.67% | test_loss: 0.5749, test_acc: 92.36%
Epoch: 500| loss: 0.0964, acc: 96.23% | test_loss: 0.3831, test_acc: 91.92%
Epoch: 600| loss: 0.0420, acc: 98.76% | test_loss: 0.7102, test_acc: 93.16%
Epoch: 700| loss: 0.0457, acc: 98.80% | test_loss: 0.3436, test_acc: 94.14%
Epoch: 800| loss: 0.0223, acc: 99.73% | test_loss: 0.5152, test_acc: 94.58%
Epoch: 900| loss: 0.0119, acc: 99.84% | test_loss: 0.6692, test_acc: 94.40%


In [162]:
tune(modelv1_2, optimizerv1_2, X2_train, y2_train, X2_test, y2_test, n_epochs=1000)

Epoch: 0| loss: 0.7282, acc: 16.94% | test_loss: 54.7010, test_acc: 83.57%
Epoch: 100| loss: 0.3789, acc: 83.06% | test_loss: 0.3736, test_acc: 83.57%
Epoch: 200| loss: 0.3132, acc: 87.79% | test_loss: 0.3487, test_acc: 87.57%
Epoch: 300| loss: 0.2935, acc: 87.88% | test_loss: 0.3520, test_acc: 86.50%
Epoch: 400| loss: 0.2788, acc: 88.19% | test_loss: 0.3800, test_acc: 87.03%
Epoch: 500| loss: 0.2591, acc: 88.83% | test_loss: 0.3662, test_acc: 84.90%
Epoch: 600| loss: 0.2493, acc: 89.61% | test_loss: 0.3710, test_acc: 87.57%
Epoch: 700| loss: 0.2309, acc: 90.05% | test_loss: 0.3792, test_acc: 87.21%
Epoch: 800| loss: 0.2336, acc: 89.72% | test_loss: 0.3916, test_acc: 86.68%
Epoch: 900| loss: 0.2003, acc: 91.32% | test_loss: 0.4154, test_acc: 87.74%


In [167]:
modelv0.to(device)

NNClassifier(
  (layer1): Linear(in_features=10, out_features=128, bias=True)
  (layer2): Linear(in_features=128, out_features=128, bias=True)
  (layer3): Linear(in_features=128, out_features=1, bias=True)
)

In [169]:
from torchmetrics.classification import BinaryAUROC
def AUCROC_metric(model, x_train, y_train, x_test, y_test):
    with torch.inference_mode():
        train_logits = model(x_train).squeeze()
        train_pred = torch.round(torch.sigmoid(train_logits))

        test_logits = model(x_test).squeeze()
        test_pred = torch.round(torch.sigmoid(test_logits)) 

    metric = BinaryAUROC(thresholds=None)
    aucroc = {"Train":metric(train_pred, y_train),
              "Test":metric(test_pred, y_test)}
    return aucroc

In [170]:
print("version 1: ",AUCROC_metric(modelv0, X1_train,y1_train, X1_test, y1_test))
print(AUCROC_metric(modelv0_0, X1_train,y1_train, X1_test, y1_test))
print(AUCROC_metric(modelv0_1 ,X1_train,y1_train, X1_test, y1_test))
print("version 2: ", AUCROC_metric(modelv1, X2_train,y2_train, X2_test, y2_test))
print(AUCROC_metric(modelv1_1, X2_train,y2_train, X2_test, y2_test))
print(AUCROC_metric(modelv1_2, X2_train,y2_train, X2_test, y2_test))


version 1:  {'Train': tensor(0.6790, device='cuda:0'), 'Test': tensor(0.6834, device='cuda:0')}
{'Train': tensor(0.8179, device='cuda:0'), 'Test': tensor(0.7693, device='cuda:0')}
{'Train': tensor(0.9983, device='cuda:0'), 'Test': tensor(0.8884, device='cuda:0')}
version 2:  {'Train': tensor(0.6176, device='cuda:0'), 'Test': tensor(0.6158, device='cuda:0')}
{'Train': tensor(0.9976, device='cuda:0'), 'Test': tensor(0.8398, device='cuda:0')}
{'Train': tensor(0.8134, device='cuda:0'), 'Test': tensor(0.7237, device='cuda:0')}
